# Spike Sorting Pipeline

**Author:** Rahim Hashim<br>
**Date:** May 2024<br>
**Description:** Pipeline to run SpikeGLX preprocessing (CatGT, TPrime, Extract Waveforms) and Kilosort4.<br>

***
## Import and Install Libraries
Make sure you have a fresh environment that contains kilosort. If not, create a new environment (`conda create env --name kilosort`) and install kilosort (`pip install kilosort`). 

In [1]:
import os
import re
import sys
import time
import warnings
import argparse
import subprocess
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib import gridspec, rcParams
warnings.simplefilter("ignore")
# install kilosort
# !pip install kilosort
from kilosort import run_kilosort

# sys.path.append('/content/drive/MyDrive/Columbia/Salzman/Projects/Spike-Sorting/Kilosort4-SGLX')

***
## Find Recording Folders

Set `root` to be where the output of a SpikeGLX recording session is (i.e. the parent directory of gandalf_20240515_g0), and specify `monkey`, `date`, and `session_num` appropriately.

In [14]:
ks_path = os.getcwd()

def read_recording_folder(root, monkey, date, session_num=0):
	sglx_folder = os.path.join(root, f'{monkey}_{date}_g{session_num}')
	imec_folder_dict = defaultdict(str)
	print(f'SpikeGLX folder: {sglx_folder}')
	# check if it exists
	if not os.path.exists(sglx_folder):
		sys.exit(f'{sglx_folder} does not exist')
	# print all folders that have imec<int> in them
	for root, dirs, files in os.walk(sglx_folder):
		for folder in dirs:
			# only if the directory ends with imec<int>
			if re.search('imec\d$', folder):
				imec_num = re.search('imec\d$', folder).group()
				imec_folder_dict[imec_num] = os.path.join(root, folder)
	if not imec_folder_dict:
		try:
			print(os.listdir(root))
		except:
			print(f'{root} missing')
		sys.exit(f'No imec folders found in {sglx_folder}')
	# order the dictionary
	imec_folder_dict = dict(sorted(imec_folder_dict.items(), key=lambda item: item[1]))
	return sglx_folder, imec_folder_dict

# root='/content/drive/Othercomputers/Ephys/E:'
# root = 'E:'
root = "C:/Users/Alejandro/Downloads"
monkey = 'gandalf'
date = '20240515'
session_num = 0
sglx_folder, imec_folder_dict = read_recording_folder(root, monkey, date, session_num)

imec_folder_dict

SpikeGLX folder: C:/Users/Alejandro/Downloads\gandalf_20240515_g0


{'imec0': 'C:/Users/Alejandro/Downloads\\gandalf_20240515_g0\\gandalf_20240515_g0_imec0'}

***
## Run CatGT

For running Importantly, the `-no_tshift` flag allows you to run CatGT without actually performing any of other operations.

In [15]:
run_catgt = True
cat_prb_fld = '0:2'
catgt_path = os.path.join(ks_path, 'CatGT-win')

if run_catgt:
	sys.path.append(catgt_path)
	if not os.path.exists(catgt_path):
		print(f'CatGT Missing: ')
	print(f"Running CatGT on {sglx_folder}")
	# time how long it takes
	start_time = time.time()
	# to run CatGT and generate tcat files with gblcar and 
	# catgt_command = f"runit.bat -dir={root} -run={monkey}_{date} -prb_fld -g={session_num} -t=0 -ni -prb={cat_prb_fld} -ap "
	# to extract sync signals and not run CatGT 
	catgt_command = f"runit.bat -dir={root} -run={monkey}_{date} -prb_fld -g={session_num} -t=0 -ni -prb={cat_prb_fld} -ap -no_tshift "
	print(f"  Bash command: {catgt_command}")
	subprocess.run(catgt_command, cwd=catgt_path, shell=True)
	print(f"  CatGT complete. Time elapsed: {time.time() - start_time:.2f} seconds")


Running CatGT on C:/Users/Alejandro/Downloads\gandalf_20240515_g0
  Bash command: runit.bat -dir=C:/Users/Alejandro/Downloads -run=gandalf_20240515 -prb_fld -g=0 -t=0 -ni -prb=0 -ap -no_tshift 
  CatGT complete. Time elapsed: 62.27 seconds


***
## Run Kilosort

In [16]:
def run_kilosort4(imec_num, settings, results):
	default_kilosort_settings = {'n_chan_bin': 385}
	try:
		ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate = \
				run_kilosort(settings=default_kilosort_settings,
										probe_name=settings['probe_name'],
										filename=settings['filename'],
										results_dir=settings['results_dir'],
										do_CAR=settings['do_CAR'],)
		results[imec_num] = {'ops': ops,
												'st': st, 
												'clu': clu, 
												'tF': tF, 
												'Wall': Wall, 
												'similar_templates': similar_templates, 
												'is_ref': is_ref, 
												'est_contam_rate': est_contam_rate}
	except:
		print(f'Error in running Kilosort4 for {imec_num}')
	return results

map_file = os.path.join(os.getcwd(), 'configFiles/neuropixels_NHP_channel_map_linear_v1.mat')
n_channels = 385
results = defaultdict(lambda: defaultdict(dict))

for imec_num, imec_folder in imec_folder_dict.items():
	print(f'{imec_num} folder: {imec_folder}')
	data_dir = imec_folder
	save_path = os.path.join(imec_folder, f'{monkey}_{date}_g{session_num}_{imec_num}_ks4')
	bin_file = os.path.join(imec_folder, f'{monkey}_{date}_g{session_num}_t0.{imec_num}.ap.bin')
	if not os.path.exists(bin_file):
		print(f'{bin_file} does not exist')
		continue
	settings = {'filename': bin_file,
							'results_dir': save_path,
							'do_CAR': True,
							'n_channels': n_channels,
							'probe_name': map_file}
	run_kilosort4(imec_num, settings, results)

imec0 folder: C:/Users/Alejandro/Downloads\gandalf_20240515_g0\gandalf_20240515_g0\gandalf_20240515_g0_imec0
Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
Using GPU for PyTorch computations. Specify `device` to change this.
using probe neuropixels_NHP_channel_map_linear_v1.mat
Preprocessing filters computed in  12.50s; total  12.51s

computing drift
Re-computing universal templates from data.


100%|██████████| 2338/2338 [38:17<00:00,  1.02it/s]


drift computed in  2345.32s; total  2357.83s

Extracting spikes using templates
Re-computing universal templates from data.


100%|██████████| 2338/2338 [37:23<00:00,  1.04it/s]


5601216 spikes extracted in  2262.18s; total  4620.07s

First clustering


...

978 clusters found, in  314.30s; total  4934.36s

Extracting spikes using cluster waveforms


100%|██████████| 2338/2338 [17:18<00:00,  2.25it/s]


7069390 spikes extracted in  1039.27s; total  5973.63s

Final clustering


100%|██████████| 96/96 [06:15<00:00,  3.91s/it]


706 clusters found, in  375.49s; total  6349.13s

Merging clusters
621 units found, in  15.02s; total  6364.16s

Saving to phy and computing refractory periods
286 units found with good refractory periods

Total runtime: 6417.66s = 01:46:58 h:m:s
Error in running Kilosort4 for imec0


***
## Run TPrime

In [13]:
from pathlib import Path

run_tprime = True
sglx_tools = os.path.join(ks_path, 'SpikeGLX_Datafile_Tools/Python/DemoReadSGLXData')
sys.path.append(sglx_tools)
from readSGLX import readMeta

if run_tprime:
	tprime_struct = defaultdict(list)
	tprime_path = os.path.join(ks_path, 'TPrime-win')
	# find all files in the session folder that have the string 'tcat.nidq.xd'
	try:
		tprime_struct['nidq'] = os.path.join(sglx_folder, 
													[f for f in os.listdir(sglx_folder) if 'tcat.nidq.xd' in f][0])
	except:
			print('   WARNING: Missing tcat.nidq.xd file. TPrime will not run.')
	for imec_num, imec_folder in imec_folder_dict.items():
		meta_file = [f for f in os.listdir(imec_folder) if f.endswith('.meta')][0]
		bin_file = os.path.join(imec_folder, f'{monkey}_{date}_g{session_num}_t0.{imec_num}.ap.bin')
		print(f'Reading {bin_file}')
		bin_file_path = Path(bin_file)
		meta_file = readMeta(bin_file_path)
		print("\tConverting spike times to seconds...")
		sample_rate = float(meta_file['imSampRate'])
		print(f"\tSampling rate: {sample_rate:.2f}")
		save_path = os.path.join(imec_folder, f'{monkey}_{date}_g{session_num}_{imec_num}_ks4')
		spike_times = np.load(os.path.join(save_path, 'spike_times.npy'))
		# keep full precision
		# spike_times_sec = spike_times / sample_rate
		spike_times_sec_file = os.path.join(save_path, "spike_times_sec.txt")
		# with open(spike_times_sec_file, 'w') as f:
		# 		for spike_time in spike_times_sec:
		# 				f.write(f"{spike_time}\n")
		print(f"\t\tGenerating {spike_times_sec_file}")
		print("\t\tDone.")
		tprime_struct['sec_files'].append(spike_times_sec_file)
		# add all files that have the string 'tcat.<imec_num>.ap.xd*' in them
		tprime_struct['imec'].append(os.path.join(imec_folder,
														[f for f in os.listdir(imec_folder) if 'tcat' in f and 'ap.xd' in f][0]))
	print(f'TPrime struct: {tprime_struct}')
	print(f'Running TPrime...')
	# time how long it takes
	start_time = time.time()
	# to run TPrime and generate spike_times_adj.npy
	tprime_command = f'runit.bat -syncperiod=1.0 -tostream={tprime_struct["nidq"]}'
	if not tprime_struct['sec_files']:
		sys.exit('No spike_times_sec.txt files found')
	if not tprime_struct['imec']:
		sys.exit('No imec files found')
	for idx, imec in enumerate(tprime_struct['imec']):
		tprime_sec_final = tprime_struct['sec_files'][idx].replace('.txt', '_adj.txt')
		tprime_command += f' -fromstream={idx},{tprime_struct["imec"][idx]}'
		tprime_command += f' -events={idx},{tprime_struct["sec_files"][idx]},{tprime_sec_final}'
	# to extract sync signals and not run CatGT 
	print(f"  Bash command: {tprime_command}")
	subprocess.run(tprime_command, cwd=tprime_path, shell=True)
	print(f"  TPrime complete. Time elapsed: {time.time() - start_time:.2f} seconds")

Reading C:/Users/Alejandro/Downloads\gandalf_20240515_g0\gandalf_20240515_g0_imec0\gandalf_20240515_g0_t0.imec0.ap.bin
	Converting spike times to seconds...
	Sampling rate: 29999.98
		Generating C:/Users/Alejandro/Downloads\gandalf_20240515_g0\gandalf_20240515_g0_imec0\gandalf_20240515_g0_imec0_ks4\spike_times_sec.txt
		Done.
TPrime struct: defaultdict(<class 'list'>, {'nidq': 'C:/Users/Alejandro/Downloads\\gandalf_20240515_g0\\gandalf_20240515_g0_tcat.nidq.xd_8_0_500.txt', 'sec_files': ['C:/Users/Alejandro/Downloads\\gandalf_20240515_g0\\gandalf_20240515_g0_imec0\\gandalf_20240515_g0_imec0_ks4\\spike_times_sec.txt'], 'imec': ['C:/Users/Alejandro/Downloads\\gandalf_20240515_g0\\gandalf_20240515_g0_imec0\\gandalf_20240515_g0_tcat.imec0.ap.xd_384_6_500.txt']})
Running TPrime...
  Bash command: runit.bat -syncperiod=1.0 -tostream=C:/Users/Alejandro/Downloads\gandalf_20240515_g0\gandalf_20240515_g0_tcat.nidq.xd_8_0_500.txt -fromstream=0,C:/Users/Alejandro/Downloads\gandalf_20240515_g0\gand

In [9]:
def plot_results(settings):

	# outputs saved to results_dir
	results_dir = Path(settings['results_dir']).joinpath('kilosort4')
	ops = np.load(results_dir / 'ops.npy', allow_pickle=True).item()
	camps = pd.read_csv(results_dir / 'cluster_Amplitude.tsv', sep='\t')['Amplitude'].values
	contam_pct = pd.read_csv(results_dir / 'cluster_ContamPct.tsv', sep='\t')['ContamPct'].values
	chan_map =  np.load(results_dir / 'channel_map.npy')
	templates =  np.load(results_dir / 'templates.npy')
	chan_best = (templates**2).sum(axis=1).argmax(axis=-1)
	chan_best = chan_map[chan_best]
	amplitudes = np.load(results_dir / 'amplitudes.npy')
	st = np.load(results_dir / 'spike_times.npy')
	clu = np.load(results_dir / 'spike_clusters.npy')
	firing_rates = np.unique(clu, return_counts=True)[1] * 30000 / st.max()
	dshift = ops['dshift']

	rcParams['axes.spines.top'] = False
	rcParams['axes.spines.right'] = False
	gray = .5 * np.ones(3)

	fig = plt.figure(figsize=(10,10), dpi=100)
	grid = gridspec.GridSpec(3, 3, figure=fig, hspace=0.5, wspace=0.5)

	ax = fig.add_subplot(grid[0,0])
	ax.plot(np.arange(0, ops['Nbatches'])*2, dshift);
	ax.set_xlabel('time (sec.)')
	ax.set_ylabel('drift (um)')

	ax = fig.add_subplot(grid[0,1:])
	t0 = 0
	t1 = np.nonzero(st > ops['fs']*5)[0][0]
	ax.scatter(st[t0:t1]/30000., chan_best[clu[t0:t1]], s=0.5, color='k', alpha=0.25)
	ax.set_xlim([0, 5])
	ax.set_ylim([chan_map.max(), 0])
	ax.set_xlabel('time (sec.)')
	ax.set_ylabel('channel')
	ax.set_title('spikes from units')

	ax = fig.add_subplot(grid[1,0])
	nb=ax.hist(firing_rates, 20, color=gray)
	ax.set_xlabel('firing rate (Hz)')
	ax.set_ylabel('# of units')

	ax = fig.add_subplot(grid[1,1])
	nb=ax.hist(camps, 20, color=gray)
	ax.set_xlabel('amplitude')
	ax.set_ylabel('# of units')

	ax = fig.add_subplot(grid[1,2])
	nb=ax.hist(np.minimum(100, contam_pct), np.arange(0,105,5), color=gray)
	ax.plot([10, 10], [0, nb[0].max()], 'k--')
	ax.set_xlabel('% contamination')
	ax.set_ylabel('# of units')
	ax.set_title('< 10% = good units')

	for k in range(2):
			ax = fig.add_subplot(grid[2,k])
			is_ref = contam_pct<10.
			ax.scatter(firing_rates[~is_ref], camps[~is_ref], s=3, color='r', label='mua', alpha=0.25)
			ax.scatter(firing_rates[is_ref], camps[is_ref], s=3, color='b', label='good', alpha=0.25)
			ax.set_ylabel('amplitude (a.u.)')
			ax.set_xlabel('firing rate (Hz)')
			ax.legend()
			if k==1:
					ax.set_xscale('log')
					ax.set_yscale('log')
					ax.set_title('loglog')

	#
	probe = ops['probe']
	# x and y position of probe sites
	xc, yc = probe['xc'], probe['yc']
	nc = 16 # number of channels to show
	good_units = np.nonzero(contam_pct <= 0.1)[0]
	mua_units = np.nonzero(contam_pct > 0.1)[0]

	gstr = ['good', 'mua']
	for j in range(2):
			print(f'~~~~~~~~~~~~~~ {gstr[j]} units ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
			print('title = number of spikes from each unit')
			units = good_units if j==0 else mua_units
			fig = plt.figure(figsize=(12,3), dpi=150)
			grid = gridspec.GridSpec(2,20, figure=fig, hspace=0.25, wspace=0.5)

			for k in range(40):
					wi = units[np.random.randint(len(units))]
					wv = templates[wi].copy()
					cb = chan_best[wi]
					nsp = (clu==wi).sum()

					ax = fig.add_subplot(grid[k//20, k%20])
					n_chan = wv.shape[-1]
					ic0 = max(0, cb-nc//2)
					ic1 = min(n_chan, cb+nc//2)
					wv = wv[:, ic0:ic1]
					x0, y0 = xc[ic0:ic1], yc[ic0:ic1]

					amp = 4
					for ii, (xi,yi) in enumerate(zip(x0,y0)):
							t = np.arange(-wv.shape[0]//2,wv.shape[0]//2,1,'float32')
							t /= wv.shape[0] / 20
							ax.plot(xi + t, yi + wv[:,ii]*amp, lw=0.5, color='k')

					ax.set_title(f'{nsp}', fontsize='small')
					ax.axis('off')
			plt.show()

In [10]:
plot_results(settings[''])

KeyError: 'data_dir'

In [11]:
settings

{'filename': 'C:/Users/Alejandro/Downloads\\gandalf_20240126_g0\\gandalf_20240126_g0_imec1\\gandalf_20240126_g0_t0.imec1.ap.bin',
 'results_dir': 'C:/Users/Alejandro/Downloads\\gandalf_20240126_g0\\gandalf_20240126_g0_imec1\\gandalf_20240126_g0_t0.imec1_ks4',
 'do_CAR': True,
 'n_channels': 385,
 'probe_name': 'C:/Users/Alejandro/Documents/GitHub/Kilosort4-SGLX/configFiles/neuropixels_NHP_channel_map_linear_v1.mat'}